In [1]:
pip install h3 osmnx

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import pandas as pd
import numpy as np
import geopandas as gpd
from shapely import wkt
import holidays
import gc

In [4]:
ca_to_ct = pd.read_csv("/content/drive/MyDrive/Triple A/data/CensusTractsTIGER2010_20250711.csv", encoding='latin-1')
ca_to_ct = ca_to_ct.rename(columns={'GEOID10': 'Tract', 'COMMAREA': 'CommunityAreaNumber'})

trips_data = pd.read_csv("/content/drive/MyDrive/Triple A/data/Taxi_Trips__2024-__20250711.csv", parse_dates=['Trip Start Timestamp'])
trips_data = trips_data[trips_data['Trip Start Timestamp'] < '2025-07-01 00:00:00']

chi_weather = pd.read_csv("/content/drive/MyDrive/Triple A/data/chicago_weather.csv")
poi = pd.read_csv("/content/drive/MyDrive/Triple A/data/CHI_POI.csv")

/tmp/ipython-input-4-3887080347.py:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  trips_data = pd.read_csv("/content/drive/MyDrive/Triple A/data/Taxi_Trips__2024-__20250711.csv", parse_dates=['Trip Start Timestamp'])


In [5]:
print(f'Total Observations: {trips_data.shape[0]}')
print(f"Dropping {trips_data['Pickup Community Area'].isna().sum()} rows due to NaN Pickup Community Area")
trips_data = trips_data.dropna(subset=['Pickup Community Area'])
print(f'{trips_data.shape[0]} Observations left')

trips_data['Trip Hour'] = pd.to_datetime(trips_data['Trip Start Timestamp']).dt.floor('h')
trips_data['Pickup Community Area'] = trips_data['Pickup Community Area'].astype(int)

Total Observations: 9793802
Dropping 275650 rows due to NaN Pickup Community Area
9518152 Observations left


/tmp/ipython-input-5-181686181.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trips_data['Trip Hour'] = pd.to_datetime(trips_data['Trip Start Timestamp']).dt.floor('h')


In [6]:
area_hourly = (
    trips_data
    .groupby(['Trip Hour', 'Pickup Community Area'], observed=True)
    .size()
    .reset_index(name='trip_count')
)

In [7]:
# Prepare tract to CA lookup
ca_to_ct['CommunityAreaNumber'] = ca_to_ct['CommunityAreaNumber'].astype(int)
tract_to_ca = ca_to_ct[['Tract', 'CommunityAreaNumber']].drop_duplicates()

# Read POIs as geometry
if poi['geometry'].dtype == object:
    poi['geometry'] = poi['geometry'].apply(wkt.loads)
poi_gdf = gpd.GeoDataFrame(poi, geometry='geometry', crs='EPSG:4326')

# Read tracts as geometry
tracts = ca_to_ct[['Tract', 'the_geom']]
tracts['geometry'] = tracts['the_geom'].apply(wkt.loads)
tracts_gdf = gpd.GeoDataFrame(tracts, geometry='geometry', crs='EPSG:4326')

# spatial join POI -> tract
joined = gpd.sjoin(poi_gdf, tracts_gdf, how='inner', predicate='within')
joined = joined.merge(tract_to_ca, on='Tract', how='left')

poi_counts_ca = (
    joined
    .groupby('CommunityAreaNumber')
    .size()
    .reset_index(name='poi_count')
)

# Merge POI count to area_hourly
area_hourly = area_hourly.merge(
    poi_counts_ca, left_on='Pickup Community Area', right_on='CommunityAreaNumber', how='left'
)
area_hourly['poi_count'] = area_hourly['poi_count'].fillna(0).astype(int)
area_hourly = area_hourly.drop(columns=['CommunityAreaNumber'])

/tmp/ipython-input-7-1225667389.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tracts['geometry'] = tracts['the_geom'].apply(wkt.loads)


In [8]:
chi_weather['timestamp_local'] = pd.to_datetime(chi_weather['timestamp_local'])
chi_weather = chi_weather.drop(columns = [
    'weather', 'timestamp_utc', 'ts', 'datetime', 'date', 'slp',
    'dhi', 'dni', 'ghi', 'solar_rad', 'azimuth', 'elev_angle',
    'h_angle', 'revision_status', 'pod'
], errors='ignore')  # Some columns may not exist

chi_weather['nighttime'] = chi_weather['pod'].apply(lambda x: 1 if x == 'n' else 0) if 'pod' in chi_weather.columns else 0
chi_weather[['clouds', 'pres', 'rh', 'vis', 'wind_dir', 'nighttime']] = chi_weather[[
    'clouds', 'pres', 'rh', 'vis', 'wind_dir', 'nighttime'
]].astype(float, errors='ignore')

# Fix missing DST hours if needed
missing_ts = pd.to_datetime(['2024-03-10 02:00', '2025-03-09 02:00'])
chi_weather = chi_weather.set_index('timestamp_local').sort_index()
filled_rows = []
for ts in missing_ts:
    if ts not in chi_weather.index:
        prev_h = ts - pd.Timedelta(hours=1)
        next_h = ts + pd.Timedelta(hours=1)
        if prev_h in chi_weather.index and next_h in chi_weather.index:
            row_before = chi_weather.loc[prev_h]
            row_after  = chi_weather.loc[next_h]
            inferred = (row_before + row_after) / 2
            inferred.name = ts
            filled_rows.append(inferred)
if filled_rows:
    df_filled = pd.DataFrame(filled_rows)
    chi_weather = pd.concat([chi_weather, df_filled]).sort_index()
chi_weather = chi_weather.reset_index().rename(columns={'index':'timestamp_local'})

# Merge weather data to hourly community area data
weather = chi_weather.copy()
weather['Trip Hour'] = weather['timestamp_local'].dt.floor('H')
weather_cols = [c for c in weather.columns if c not in ('timestamp_local')]
weather_hourly = weather[weather_cols].drop_duplicates(subset='Trip Hour', keep='first')

area_hourly = area_hourly.merge(weather_hourly, on='Trip Hour', how='left')

/tmp/ipython-input-8-2854937674.py:34: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  weather['Trip Hour'] = weather['timestamp_local'].dt.floor('H')


In [9]:

us_holidays = holidays.US(state='IL', years=[2024, 2025])
area_hourly['is_holiday'] = area_hourly['Trip Hour'].dt.date.apply(lambda x: int(x in us_holidays))
area_hourly['isWeekend'] = (area_hourly['Trip Hour'].dt.dayofweek >= 5).astype(int)
area_hourly['Trip Start Day'] = area_hourly['Trip Hour'].dt.day
area_hourly['Trip Start Hour'] = area_hourly['Trip Hour'].dt.hour
area_hourly['Trip Start Month'] = area_hourly['Trip Hour'].dt.month

area_hourly["Trip Start Hour Sin"] = np.sin(2 * np.pi * area_hourly["Trip Start Hour"] / 24)
area_hourly["Trip Start Hour Cos"] = np.cos(2 * np.pi * area_hourly["Trip Start Hour"] / 24)
area_hourly["Trip Start Day Sin"] = np.sin(2 * np.pi * area_hourly["Trip Start Day"] / 7)
area_hourly["Trip Start Day Cos"] = np.cos(2 * np.pi * area_hourly["Trip Start Day"] / 7)
area_hourly["Trip Start Month Sin"] = np.sin(2 * np.pi * area_hourly["Trip Start Month"] / 12)
area_hourly["Trip Start Month Cos"] = np.cos(2 * np.pi * area_hourly["Trip Start Month"] / 12)


In [10]:
area_agg_dict = {
    'trip_count': 'sum',
    **{col: 'mean' for col in area_hourly.columns
          if col not in ['trip_count', 'Pickup Community Area', 'Trip Hour']}
}

def aggregate_area_by_freq(df, freq):
    grouped = (
        df.groupby(
            [pd.Grouper(key='Trip Hour', freq=freq), 'Pickup Community Area'],
            observed=True
        )
        .agg(area_agg_dict)
        .reset_index()
    )
    return grouped

area_2hourly  = aggregate_area_by_freq(area_hourly, '2H')
area_4hourly  = aggregate_area_by_freq(area_hourly, '4H')
area_6hourly  = aggregate_area_by_freq(area_hourly, '6H')
area_12hourly = aggregate_area_by_freq(area_hourly, '12H')

/tmp/ipython-input-10-475972850.py:10: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  [pd.Grouper(key='Trip Hour', freq=freq), 'Pickup Community Area'],
/tmp/ipython-input-10-475972850.py:10: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  [pd.Grouper(key='Trip Hour', freq=freq), 'Pickup Community Area'],
/tmp/ipython-input-10-475972850.py:10: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  [pd.Grouper(key='Trip Hour', freq=freq), 'Pickup Community Area'],
/tmp/ipython-input-10-475972850.py:10: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  [pd.Grouper(key='Trip Hour', freq=freq), 'Pickup Community Area'],


In [11]:
dfs = {'area_hourly_h3':area_hourly,
       'area_2hourly_h3':area_2hourly,
       'area_4hourly_h3':area_4hourly,
       'area_6hourly_h3':area_6hourly,
       'area_12hourly_h3':area_12hourly}

for name, df in dfs.items():
    df.to_csv(f'/content/drive/MyDrive/Triple A/data/{name}.csv', index = False)